In [2]:
# -------------- #

import pandas as pd
import os
import datetime
import math
import load_data

from config import (
    RAW_DATA_DIR,
    LAG_DICT,
    SLIDING_DICT
)

from tqdm import tqdm
import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt
color = sns.color_palette()

import subprocess
from subprocess import check_output
import gc

%matplotlib inline

In [12]:
holidays_events_df = pd.read_csv(RAW_DATA_DIR+'holidays_events.csv')
items_df = pd.read_csv(RAW_DATA_DIR+'items.csv')
oil_df = pd.read_csv(RAW_DATA_DIR+'oil.csv',parse_dates=['date'])
stores_df = pd.read_csv(RAW_DATA_DIR+'stores.csv')
transactions_df = pd.read_csv(RAW_DATA_DIR+'transactions.csv')

train_df = pd.read_csv(
    'datasets/train.csv', usecols=[1, 2, 3, 4, 5],
    parse_dates=["date"]
)

test_df = pd.read_csv(
    'datasets/test.csv', usecols=[1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]
)

/Users/liuyu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Memory Deducion

In [ ]:
#--- for integer type columns ---
def change_datatype(df):
    float_cols = list(df.select_dtypes(include=['int']).columns)
    for col in float_cols:
        if ((np.max(df[col]) <= 127) and(np.min(df[col] >= -128))):
            df[col] = df[col].astype(np.int8)
        elif ((np.max(df[col]) <= 32767) and(np.min(df[col] >= -32768))):
            df[col] = df[col].astype(np.int16)
        elif ((np.max(df[col]) <= 2147483647) and(np.min(df[col] >= -2147483648))):
            df[col] = df[col].astype(np.int32)
        else:
            df[col] = df[col].astype(np.int64)

#--- for float type columns ---
def change_datatype_float(df):
    float_cols = list(df.select_dtypes(include=['float']).columns)
    for col in float_cols:
        df[col] = df[col].astype(np.float32)            

## How many stores. How many items in each stores. Their growth

## 